In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import class_weight
from tensorflow.keras.layers import Reshape, Input, Dense, Activation, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import tensorflow

# Kokeiltavat mallit
from sklearn.neural_network import MLPClassifier


In [2]:
import tensorflow as tf
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))

Num CPUs Available:  1


In [3]:
df = pd.read_csv('data/02-14-2018.csv')
pd.set_option('display.max_columns', None)

In [4]:
df

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026633,5.632086e+07,1.393000e+02,56320958,56320761,112641719,5.632086e+07,1.393000e+02,56320958,56320761,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.00,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026633,5.632073e+07,1.145513e+02,56320814,56320652,112641466,5.632073e+07,1.145513e+02,56320814,56320652,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.00,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,0.026634,5.631931e+07,3.019346e+02,56319525,56319098,112638623,5.631931e+07,3.019346e+02,56319525,56319098,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0.026634,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0.00,0.000000,0.000000,0,0,0,0,0,0,3,0,0,0,-1,-1,0,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,82.600000,196.741237,976,0,227.300000,371.677892,544.161528,3.873587,2.689152e+05,2.474438e+05,673900,22,6453966,4.609976e+05,1.231094e+05,673900,229740,5637902,6.264336e+05,4.550822e+05,1167293,554,0,0,0,0,488,328,2.324152,1.549435,0,976,135.076923,277.834760,77192.153846,0,0,0,1,0,0,0,0,0,140.48,82.600000,227.300000,0,0,0,0,0,0,15,1239,10,2273,65535,233,6,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,81.642857,203.745545,976,0,200.818182,362.249864,380.733175,2.839597,3.668361e+05,5.113566e+05,1928102,21,8804066,6.772358e+05,5.324170e+05,1928102,246924,7715481,7.715481e+05,7.555431e+05,2174893,90,0,0,0,0,456,360,1.590174,1.249423,0,976,128.923077,279.763032,78267.353846,0,0,0,1,0,0,0,0,0,134.08,81.642857,200.818182,0,0,0,0,0,0,14,1143,11,2209,5808,233,6,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,80,6,14/02/2018 10:53:23,10156986,5,5,1089,1923,587,0,217.800000,299.745225,1334,0,384.600000,588.831725,296.544664,0.984544,1.128554e+06,3.290455e+06,9902474,17,254512,6.362800e+04,6.417719e+04,123760,486,10140598,2.535150e+06,4.976408e+06,9999381,6350,0,0,0,0,112,104,0.492272,0.492272,0,1334,273.818182,435.695953,189830.963636,0,0,0,1,0,0,0,0,1,301.20,217.800000,384.600000,0,0,0,0,0,

In [5]:
df.describe()

C:\Users\Tooslickki\miniconda3\envs\kybs_ht\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\Users\Tooslickki\miniconda3\envs\kybs_ht\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.046298e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1048575.0,1048575.0,1048575.0,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1048575.0,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1048575.0,1048575.0,1048575.0,1048575.0,1048575.0,1048575.0,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,4.876262e+03,8.107557e+00,6.255555e+06,6.206622e+00,7.211191e+00,4.479936e+02,4.521803e+03,1.745736e+02,8.389535e+00,3.879579e+01,5.021107e+01,3.325241e+02,2.016587e+01,9.275989e+01,1.113899e+02,inf,inf,1.583948e+06,2.101497e+06,6.306042e+06,-1.519513e+06,5.944303e+06,1.775583e+06,2.147776e+06,6.081985e+06,-1.475314e+06,5.754843e+06,4.798110e+05,6.049147e+05,1.770279e+06,8.880474e+04,2.659419e-02,0.0,0.0,0.0,1.560079e+02,1.727050e+02,1.413886e+05,1.258575e+05,8.322709e+00,3.375417e+02,6.311424e+01,9.570919e+01,3.092890e+04,3.572467e-03,2.659419e-02,3.960422e-02,5.254312e-01,2.634699e-01,1.211463e-01,0.0,3.960327e-02,6.634537e-01,7.144445e+01,3.879579e+01,9.275989e+01,0.0,0.0,0.0,0.0,0.0,0.0,6.206622e+00,4.479936e+02,7.211191e+00,4.521803e+03,1.050166e+04,6.142295e+03,2.793536e+00,2.327970e+01,5.152449e+04,2.136151e+04,8.789157e+04,3.995477e+04,3.101206e+06,7.297218e+05,4.812391e+06,2.126920e+06
std,1.444344e+04,4.460625e+00,1.260291e+09,4.447851e+01,1.048682e+02,1.573541e+04,1.515021e+05,2.876713e+02,1.948279e+01,5.331882e+01,8.518887e+01,4.938587e+02,4.653823e+01,1.579997e+02,1.860082e+02,NaN,NaN,3.332477e+08,7.332938e+08,1.530125e+09,1.640806e+09,1.260287e+09,3.332505e+08,7.332951e+08,1.530123e+09,1.640806e+09,2.257935e+07,2.291242e+06,2.612809e+06,7.985050e+06,1.690612e+06,1.608943e-01,0.0,0.0,0.0,1.078119e+03,2.103964e+03,3.085265e+05,2.670306e+05,1.890669e+01,5.023828e+02,1.007340e+02,1.475421e+02,1.170697e+05,5.966329e-02,1.608943e-01,1.950276e-01,4.993531e-01,4.405153e-01,3.262974e-01,0.0,1.950254e-01,5.652833e-01,1.051520e+02,5.331882e+01,1.579997e+02,0.0,0.0,0.0,0.0,0.0,0.0,4.447851e+01,1.573541e+04,1.048682e+02,1.515021e+05,1.293709e+04,1.814980e+04,5.557106e+00,1.106185e+01,5.815586e+05,2.186405e+05,7.395725e+05,5.602693e+05,5.414780e+08,3.820031e+08,1.522117e+09,1.817013e+07
min,0.000000e+00,0.000000e+00,-9.190110e+11,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00000

Datan esikäsittely

In [6]:
# Tiputetaan tarpeettomat kolumnit (sisältävät pelkästään arvoja 0)
df = df.drop(columns=['Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'CWE Flag Count', 'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg', 'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg'])

In [7]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
df.dropna(inplace=True)

In [9]:
df.drop(columns=['Timestamp'], inplace=True)

In [10]:
columns_to_drop = ['Dst Port', 'Protocol']
df.drop(columns=columns_to_drop, inplace=True)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1044751 entries, 0 to 1048574
Data columns (total 67 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Flow Duration      1044751 non-null  int64  
 1   Tot Fwd Pkts       1044751 non-null  int64  
 2   Tot Bwd Pkts       1044751 non-null  int64  
 3   TotLen Fwd Pkts    1044751 non-null  int64  
 4   TotLen Bwd Pkts    1044751 non-null  int64  
 5   Fwd Pkt Len Max    1044751 non-null  int64  
 6   Fwd Pkt Len Min    1044751 non-null  int64  
 7   Fwd Pkt Len Mean   1044751 non-null  float64
 8   Fwd Pkt Len Std    1044751 non-null  float64
 9   Bwd Pkt Len Max    1044751 non-null  int64  
 10  Bwd Pkt Len Min    1044751 non-null  int64  
 11  Bwd Pkt Len Mean   1044751 non-null  float64
 12  Bwd Pkt Len Std    1044751 non-null  float64
 13  Flow Byts/s        1044751 non-null  float64
 14  Flow Pkts/s        1044751 non-null  float64
 15  Flow IAT Mean      1044751 non-null  

In [12]:
df["Label"].value_counts()

Label
Benign            663808
FTP-BruteForce    193354
SSH-Bruteforce    187589
Name: count, dtype: int64

In [13]:
df.replace(to_replace=["FTP-BruteForce", "SSH-Bruteforce"], value="Attack", inplace=True)

In [14]:
df.replace(to_replace="Benign", value=0, inplace=True)
df.replace(to_replace="Attack", value=1, inplace=True)

In [15]:
train, test = train_test_split(df, test_size=0.3)

In [16]:
min_max_scaler = MinMaxScaler().fit(train[train.drop(columns=["Label"]).columns])

In [17]:
train[train.drop(columns=["Label"]).columns] = min_max_scaler.transform(train[train.drop(columns=["Label"]).columns])

In [18]:
test[test.drop(columns=["Label"]).columns] = min_max_scaler.transform(test[test.drop(columns=["Label"]).columns])

In [19]:
y_train = np.array(train.pop("Label"))
X_train = train.values

y_test = np.array(test.pop("Label"))
X_test = test.values

In [65]:
print(X_train.shape)
print(y_train.shape)

(731325, 66)
(731325,)


In [21]:
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

In [22]:
result = clf.predict(X_test)

In [23]:
np.unique(result, return_counts=True)

(array([0, 1], dtype=int64), array([199311, 114115], dtype=int64))

In [24]:
np.unique(y_test, return_counts=True)

(array([0, 1], dtype=int64), array([199313, 114113], dtype=int64))

In [25]:
np.set_printoptions(threshold=np.inf)

In [26]:
differing_values = np.sum(result != y_test)

In [27]:
differing_values

16

In [28]:
1 - differing_values / len(result)

0.9999489512676039

In [66]:
y_train_small  = y_train[0:1000]
X_train_small = X_train[0:1000]

y_test_small  = y_test[0:1000]
X_test_small  = X_test[0:1000]

##y_train_small = tensorflow.keras.utils.to_categorical(y_train_small, 1)
##y_test_small = tensorflow.keras.utils.to_categorical(y_test_small, 1)

print(y_train_small.shape)
print(X_train_small.shape)
print(y_test_small.shape)
print(X_test_small.shape)

(1000,)
(1000, 66)
(1000,)
(1000, 66)


In [70]:
X_train.shape[1]

66

In [77]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["Label"])

# Convert labels to one-hot encoding
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

# Build the CNN model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu')) #128
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=256, validation_data=(X_test, y_test_one_hot))

Epoch 1/10
2857/2857 [==============================] - 39s 12ms/step - loss: 0.0106 - accuracy: 0.9970 - val_loss: 0.0014 - val_accuracy: 0.9994
Epoch 2/10
2857/2857 [==============================] - 32s 11ms/step - loss: 0.0021 - accuracy: 0.9994 - val_loss: 5.5101e-04 - val_accuracy: 0.9999
Epoch 3/10
2857/2857 [==============================] - 33s 11ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 5.2653e-04 - val_accuracy: 0.9999
Epoch 4/10
2857/2857 [==============================] - 33s 11ms/step - loss: 9.5133e-04 - accuracy: 0.9998 - val_loss: 3.9433e-04 - val_accuracy: 0.9999
Epoch 5/10
2857/2857 [==============================] - 33s 12ms/step - loss: 7.9226e-04 - accuracy: 0.9998 - val_loss: 3.9935e-04 - val_accuracy: 0.9999
Epoch 6/10
2419/2857 [========================>.....] - ETA: 4s - loss: 6.2639e-04 - accuracy: 0.9999

KeyboardInterrupt: 

Sitten CNN

In [67]:
def general_CNN():
    model = Sequential()
    
    # Add a Reshape layer to add a channel dimension
    model.add(Reshape((66, 1), input_shape=(66,)))

    model.add(Conv1D(64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(128, kernel_size=3, activation='relu'))
    
    model.add(Flatten())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [68]:
with tf.device('/gpu:0'):
    deep_cnn_model = general_CNN()
print(deep_cnn_model.summary()) 

from tensorflow.keras.utils import plot_model
plot_model(deep_cnn_model, to_file='deep_cnn_model.png',show_shapes=True)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_4 (Reshape)         (None, 66, 1)             0         
                                                                 
 conv1d_9 (Conv1D)           (None, 64, 64)            256       
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 32, 64)            0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, 30, 128)           24704     
                                                                 
 flatten_4 (Flatten)         (None, 3840)              0         
                                                                 
 dense_8 (Dense)             (None, 256)               983296    
                                                      

In [69]:
history = deep_cnn_model.fit(X_train_small, y_train_small, 
                   epochs=100, 
                   batch_size=25,
                   shuffle=True,
                   validation_data=(X_test_small,y_test_small))

Epoch 1/100
40/40 [==============================] - 11s 141ms/step - loss: 0.0000e+00 - accuracy: 0.3550 - val_loss: 0.0000e+00 - val_accuracy: 0.3730
Epoch 2/100
40/40 [==============================] - 4s 115ms/step - loss: 0.0000e+00 - accuracy: 0.3550 - val_loss: 0.0000e+00 - val_accuracy: 0.3730
Epoch 3/100
40/40 [==============================] - 3s 67ms/step - loss: 0.0000e+00 - accuracy: 0.3550 - val_loss: 0.0000e+00 - val_accuracy: 0.3730
Epoch 4/100
40/40 [==============================] - 3s 67ms/step - loss: 0.0000e+00 - accuracy: 0.3550 - val_loss: 0.0000e+00 - val_accuracy: 0.3730
Epoch 5/100
40/40 [==============================] - 3s 67ms/step - loss: 0.0000e+00 - accuracy: 0.3550 - val_loss: 0.0000e+00 - val_accuracy: 0.3730
Epoch 6/100
40/40 [==============================] - 3s 67ms/step - loss: 0.0000e+00 - accuracy: 0.3550 - val_loss: 0.0000e+00 - val_accuracy: 0.3730
Epoch 7/100
40/40 [==============================] - 3s 67ms/step - loss: 0.0000e+00 - accuracy: 

KeyboardInterrupt: 